# Carregamento de pacotes

In [1]:
import pandas as pd
import warnings

# Configuração do notebook

In [2]:
warnings.filterwarnings('ignore')

# Funções

In [3]:
# Remove o modelo da descrição se estiver repetido
def strip_string_with_prefix_from_text(text, prefix, string):
    text = text.strip()
    prefix = prefix.strip()
    string = string.replace(' ', '')

    length_prefix = len(prefix)
    length_string = len(string)

    if text[0:length_prefix] == prefix:
        text = text[length_prefix:].strip()
    
    string_in_text = text[0:length_string]

    if string_in_text == string:
        return text[length_string + 1:].strip()
    else:
        return text

# Identifica se há indicação de ser hatch ou sedan na descrição e trata o nome do modelo de acordo
def is_hatch_or_sedan(description, model):
    hatch = 'HATCH'
    sedan = 'SEDAN'
    fastback = 'FASTBACK' # Trata exceção do focus
    if description.upper().find(hatch) != -1:
        return model + ' ' + hatch
    elif description.upper().find(sedan) != -1:
        return model + ' ' + sedan
    elif description.upper().find(fastback) != -1 and model == 'FOCUS':
        return model + ' ' + sedan
    else:
        return model
    
# Identifica o tipo de transmissão
def label_transmisson(description, names, label):
    for name in names:
        if description.find(name) != -1:
            return label

def merge_transmission_cols(label1, label2, label3):
    if label1 != None:
        return label1
    elif label2 != None:
        return label2
    else:
        return label3

def write_to_file(file_name, content):
    f = open(file_name, 'w+')
    for line in content:
        f.write(f'{line}\n')
    f.close()

# Carregamento da base de dados

In [4]:
df_full = pd.read_csv('Extração Zoss 30.07.21.csv')
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92291 entries, 0 to 92290
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CodigoFipe           84560 non-null  object 
 1   Marca                92291 non-null  object 
 2   Modelo               92291 non-null  object 
 3   AnoModelo            92291 non-null  int64  
 4   AnoFabricacao        92291 non-null  int64  
 5   RegraKM              92291 non-null  object 
 6   Quilometragem        92291 non-null  int64  
 7   PrecoAnuncio         92291 non-null  float64
 8   TipoAnuncio          92291 non-null  object 
 9   DescricaoSite        92291 non-null  object 
 10  Cidade               92291 non-null  object 
 11  Estado               92291 non-null  object 
 12  Pais                 92291 non-null  object 
 13  MarketPlace          86080 non-null  object 
 14  Vendedor             92291 non-null  object 
 15  vendedorMarketplace  92291 non-null 

In [5]:
# Amostragem
df = df_full
# df = df_full.head(1000)
# df = df_full.sample(1000)

# Tratamento da coluna Modelo
* Foi identificado que há duplicidade com o HB20S, tendo ocorrências como 'HB20S' e 'HB20 S'
* O Onix Joy está identificado como Joy apenas

In [6]:
# Correção da identificação do HB20S
df.replace('HB20 S', 'HB20S', inplace=True)

In [7]:
# Corrige a identificação do Onix Joy
df['DescricaoSite'] = df['DescricaoSite'].apply(lambda row: 'JOY ' + row if row == 'JOY' else row)
df['Modelo'].replace('JOY', 'ONIX', inplace=True)

# Tratamento da coluna de descrição
* Algumas entradas possuiam o nome do modelo do carro repetido
* Alguns nome de modelo tinham o prefixo 'NEW' e este foi desconsiderado

In [8]:
# Retira o modelo da descrição
df['Descricao'] = df.apply(lambda row: strip_string_with_prefix_from_text(row['DescricaoSite'], 'NEW ', row['Modelo']), axis=1)

print(df.shape)
df.head()

(92291, 20)


,CodigoFipe,Marca,Modelo,AnoModelo,AnoFabricacao,RegraKM,Quilometragem,PrecoAnuncio,TipoAnuncio,DescricaoSite,Cidade,Estado,Pais,MarketPlace,Vendedor,vendedorMarketplace,IdAnuncio,LinkAnuncio,DATA,Descricao
0,NaN,KIA,SPORTAGE,2020,2019,ATÉ 15 (KM),1628,171480.0,PJ,NEW SPORTAGE EX 2.0 FLEX 4P C/AR - AUTOMATICO,SAO PAULO,SP,BR,NaN,LOCALIZA,Localiza,102522,https://seminovos.localiza.com/kia_sportage_ex...,2021-07-30,EX 2.0 FLEX 4P C/AR - AUTOMATICO
1,NaN,FORD,ECOSPORT,2020,2019,ATÉ 15 (KM),5940,81450.0,PJ,ECOSPORT SE 1.5 12V FLEX 4P C/AR - AUTOMATICO,SAO PAULO,SP,BR,NaN,LOCALIZA,Localiza,110397,https://seminovos.localiza.com/ford_ecosport_s...,2021-07-30,SE 1.5 12V FLEX 4P C/AR - AUTOMATICO
2,NaN,FORD,ECOSPORT,2020,2019,ATÉ 15 (KM),6830,81070.0,PJ,ECOSPORT SE 1.5 12V FLEX 4P C/AR - AUTOMATICO,OSASCO,SP,BR,NaN,LOCALIZA,Localiza,108256,https://seminovos.localiza.com/ford_ecosport_s...,2021-07-30,SE 1.5 12V FLEX 4P C/AR - AUTOMATICO
3,NaN,BMW,X4,2017,2016,ATÉ 15 (KM),7454,295910.0,PJ,X4 XDRIVE 35I M SPORT 3.0 4P C/AR - AUTOMATICO...,SAO PAULO,SP,BR,NaN,LOCALIZA,Localiza,105992,https://seminovos.localiza.com/bmw_x4_xdrive_2...,2021-07-30,XDRIVE 35I M SPORT 3.0 4P C/AR - AUTOMATICO BL...
4,NaN,JEEP,RENEGADE,2019,2019,ATÉ 15 (KM),7496,94060.0,PJ,RENEGADE LONGITUDE 1.8 16V FLEX 4P C/AR - AUTO...,NITEROI,RJ,BR,NaN,LOCALIZA,Localiza,110387,https://seminovos.localiza.com/jeep_renegade_l...,2021-07-30,LONGITUDE 1.8 16V FLEX 4P C/AR - AUTOMATICO


# Identificação das características dos modelos
* Alguns modelos possuem diferenciação entre hatch e sedan
* Volume do motor
* Tipo de transmissão
* Diesel ou não

In [9]:
# Identifica se é hatch ou sedan pela descrição
df['Modelo'] = df.apply(lambda row: is_hatch_or_sedan(row['DescricaoSite'], row['Modelo']), axis=1)

In [10]:
# Identifica como hatch os que tem versão sedan e não são sedan
models = list(pd.Series(df['Modelo'].unique()).values)
for model in models:
    if model + ' SEDAN' in models:
        df['Modelo'].replace(model, model + ' HATCH', inplace=True)

In [11]:
# Retira a identificação de hatch dos que não tem versão sedan
models = list(pd.Series(df['Modelo'].unique()).values)
for model in models:
    model_unhatched = model.replace(' HATCH', '')
    if (' HATCH' in model) & (model_unhatched + ' SEDAN' not in models):
        df['Modelo'].replace(model, model_unhatched, inplace=True)

In [12]:
# Identificação do volume do motor
df['VolumeMotor'] = df.apply(lambda row: row['Descricao'][row['Descricao'].find('.') - 1 : row['Descricao'].find('.') + 2], axis=1)
df['VolumeMotor'].replace(r'', 'Eletrico', inplace=True)

In [13]:
# Identifica o tipo de transmissão
good_automatics = ['AUTOMATICO','DIRECT SHIFT', '4MOTION DSG', 'GEARTRONIC', 'MULTIDRIVE', '9G-TRONIC', 'X-TRONIC', 
                   'S TRONIC','S-TRONIC', '7G-DCT', '4MATIC', 'AT6', 'AT9', 'CVT', 'DCT', 'DSG']
                   
bad_automatics = ['AUTOMATIZADO', 'POWERSHIFT', 'TIPTRONIC', 'I-MOTION', 'EASY-R']

df['TransmissaoATBoa'] = df.apply(lambda row: label_transmisson(row['Descricao'], good_automatics, 'AUTOMATICO BOM'), axis=1)
df['TransmissaoATRuim'] = df.apply(lambda row: label_transmisson(row['Descricao'], bad_automatics, 'AUTOMATICO RUIM'), axis=1)
df['Trasmissao'] = df.apply(lambda row: merge_transmission_cols(row['TransmissaoATBoa'], row['TransmissaoATRuim'], 'MANUAL'), axis=1)
df.drop(['TransmissaoATBoa'], axis=1, inplace=True)
df.drop(['TransmissaoATRuim'], axis=1, inplace=True)

In [14]:
# Identificação se o motor é Diesel ou não
df['Combustivel'] = df['Descricao'].apply(lambda row: 'DIESEL' if 'DIESEL' in row.upper()
                                                else ('ELETRICO' if 'ELETRICO' in row.upper()
                                                else  'OUTRO'))

# Adequação dos dados ao uso

In [15]:
# Eliminação das colunas que não serão usadas
df.drop(['CodigoFipe'], axis=1, inplace=True)
df.drop(['RegraKM'], axis=1, inplace=True)
df.drop(['DescricaoSite'], axis=1, inplace=True)
df.drop(['IdAnuncio'], axis=1, inplace=True)
df.drop(['DATA'], axis=1, inplace=True) #Todas as datas são iguais
df.drop(['LinkAnuncio'], axis=1, inplace=True)

# Visualização e armazenamento da base tratada

In [16]:
print(df.shape)
df.head()

(92291, 17)


,Marca,Modelo,AnoModelo,AnoFabricacao,Quilometragem,PrecoAnuncio,TipoAnuncio,Cidade,Estado,Pais,MarketPlace,Vendedor,vendedorMarketplace,Descricao,VolumeMotor,Trasmissao,Combustivel
0,KIA,SPORTAGE,2020,2019,1628,171480.0,PJ,SAO PAULO,SP,BR,NaN,LOCALIZA,Localiza,EX 2.0 FLEX 4P C/AR - AUTOMATICO,2.0,AUTOMATICO BOM,OUTRO
1,FORD,ECOSPORT,2020,2019,5940,81450.0,PJ,SAO PAULO,SP,BR,NaN,LOCALIZA,Localiza,SE 1.5 12V FLEX 4P C/AR - AUTOMATICO,1.5,AUTOMATICO BOM,OUTRO
2,FORD,ECOSPORT,2020,2019,6830,81070.0,PJ,OSASCO,SP,BR,NaN,LOCALIZA,Localiza,SE 1.5 12V FLEX 4P C/AR - AUTOMATICO,1.5,AUTOMATICO BOM,OUTRO
3,BMW,X4,2017,2016,7454,295910.0,PJ,SAO PAULO,SP,BR,NaN,LOCALIZA,Localiza,XDRIVE 35I M SPORT 3.0 4P C/AR - AUTOMATICO BL...,3.0,AUTOMATICO BOM,OUTRO
4,JEEP,RENEGADE,2019,2019,7496,94060.0,PJ,NITEROI,RJ,BR,NaN,LOCALIZA,Localiza,LONGITUDE 1.8 16V FLEX 4P C/AR - AUTOMATICO,1.8,AUTOMATICO BOM,OUTRO


In [17]:
# Salva arquivo com a base tratada
df.to_csv('base_tratada.csv')